In [1]:
#Instalando o wandb
!!pip install wandb -qqq

['\x1b[?25l',
 '\x1b[K     |▏                               | 10kB 22.7MB/s eta 0:00:01',
 '\x1b[K     |▎                               | 20kB 14.6MB/s eta 0:00:01',
 '\x1b[K     |▌                               | 30kB 9.6MB/s eta 0:00:01',
 '\x1b[K     |▋                               | 40kB 7.8MB/s eta 0:00:01',
 '\x1b[K     |▉                               | 51kB 6.7MB/s eta 0:00:01',
 '\x1b[K     |█                               | 61kB 7.2MB/s eta 0:00:01',
 '\x1b[K     |█▏                              | 71kB 7.5MB/s eta 0:00:01',
 '\x1b[K     |█▎                              | 81kB 7.6MB/s eta 0:00:01',
 '\x1b[K     |█▌                              | 92kB 8.0MB/s eta 0:00:01',
 '\x1b[K     |█▋                              | 102kB 7.5MB/s eta 0:00:01',
 '\x1b[K     |█▊                              | 112kB 7.5MB/s eta 0:00:01',
 '\x1b[K     |██                              | 122kB 7.5MB/s eta 0:00:01',
 '\x1b[K     |██                              | 133kB 7.5MB/s eta 0:00:01',
 '\x1

In [2]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
#Importando bibliotecas
import wandb
import keras 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.datasets import mnist
from wandb.keras import WandbCallback


In [4]:
#Importando dataset e preprocessando dados

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('x_train shape => {}'.format(x_train.shape))
print('x_test shape => {}'.format(x_test.shape))

x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

print('x_train shape => {}'.format(x_train.shape))
print('x_test shape => {}'.format(x_test.shape))

x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255




11493376/11490434 [==============================] - 0s 0us/step
x_train shape => (60000, 28, 28)
x_test shape => (10000, 28, 28)
x_train shape => (60000, 784)
x_test shape => (10000, 784)


In [5]:
def train():
  #Ajusta configuracoes do wandb
  wandb.init(project="teste_mnist_overfitting", config={
    "num_layers" : 2,
    "layer_1": 512,
    "optimizer": "adam",
    "learning_rate" : 0.01,
    "batch_size" : 16,
    "dropout" : 0.0,
    "epochs": 30,
    "patience" : 10,
    "architecture": "NLP",
    "dataset": "MNIST",
    })
  config = wandb.config


  (x_train, y_train), (x_test, y_test) = mnist.load_data()

  x_train = x_train.reshape(x_train.shape[0], -1)
  x_test = x_test.reshape(x_test.shape[0], -1)

  x_train = x_train.astype('float32')/255
  x_test = x_test.astype('float32')/255



  #Cria o modelo
  model = Sequential()

  for i in range(config.num_layers):
    model.add(Dense(units = config.layer_1, activation = 'relu', kernel_initializer = 'random_uniform'))
    model.add(Dropout(config.dropout))

  model.add(Dense(units=10, activation= 'softmax'))

  #Compila o modelo
  model.compile(optimizer= config.optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

  #Ajusta o modelo aos dados
  history = model.fit(x = x_train,
                    y = y_train,
                    epochs = config.epochs,
                    batch_size = config.batch_size,
                    validation_data = (x_test, y_test),
                    callbacks = [WandbCallback(), EarlyStopping(patience = config.patience)]
                    )
  

In [7]:
#Configurando os Sweeps

#Metodo de sweep
sweep_config = {
      'method': 'random',
      'metric': {
          'name': 'val_accuracy',
          'goal': 'maximize'
      }
}

#Parametros usados no sweep
parameters_dict = {
        'optimizer': {
            'value': 'adam'
              },
       'num_layers': {
            'max': 6,
            'min': 2,
            'distribution': 'int_uniform',
        },
        'layer_1': {
            'max': 1024,
            'min': 512,
            'distribution': 'int_uniform',
        },
        'epochs' : {
            'value' : 30
            },
        'batch_size' : {
            'values' : [128, 256, 512]
           },
        'learning_rate': {
            'min': -4,
            'max': -1,
            'distribution': 'log_uniform',  
        },
        'dropout' : {
           'min': 0.0,
           'max': 0.5,
            'distribution': 'uniform', 
        },
        'patience' : {
            'values' : [30, 3]
        }
}

sweep_config['parameters'] = parameters_dict

#cria o sweep
sweep_id = wandb.sweep(sweep_config,project="random_final_ajustado")

#Executa o sweep
wandb.agent(sweep_id, function=train, count = 100)


469/469 [==============================] - 3s 6ms/step - loss: 0.0309 - accuracy: 0.9917 - val_loss: 0.1040 - val_accuracy: 0.9804
Epoch 22/30
469/469 [==============================] - 3s 6ms/step - loss: 0.0378 - accuracy: 0.9909 - val_loss: 0.1097 - val_accuracy: 0.9836
Epoch 23/30
  1/469 [..............................] - ETA: 2s - loss: 0.0393 - accuracy: 0.9844

wandb: Ctrl + C detected. Stopping sweep.
